In [1]:
%%HTML
<style>
.container { width: 100% }
</style>

# Converting a Non-Deterministic <span style="font-variant:small-caps;">Fsm</span> into a Deterministic <span style="font-variant:small-caps;">Fsm</span>

In this section we show how a non-deterministic <span style="font-variant:small-caps;">Fsm</span>
$$ A = \langle Q, \Sigma, \delta, q_0, F \rangle $$
can be transformed into a deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(A)$ such that both 
<span style="font-variant:small-caps;">Fsm</span>s accept the
same language, that is we have
$$ L(A) = L\bigl(\texttt{det}(A)\bigr). $$
The idea behind this transformation is that the <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(A)$ has to 
compute the set of all states that the <span style="font-variant:small-caps;">Fsm</span> $A$ could be in. 
Hence the states of the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(A)$ are 
**sets** of states of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $A$.  A set of these states contains all those states that the non-deterministic <span style="font-variant:small-caps;">Fsm</span> 
$A$ could have reached.  Furthermore, a set $M$ of states of the <span style="font-variant:small-caps;">Fsm</span> $A$ is an accepting state of the <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(A)$ if the set $M$ contains an accepting state of the <span style="font-variant:small-caps;">Fsm</span> $A$.

In [2]:
%run FixedPoint.ipynb

In order to present the construction of $\texttt{det}(A)$ we first have to define two auxiliary functions.
We start with the <em style="color:blue">$\varepsilon$-closure</em> of a given state. 

The function `epsClosure` takes two arguments:
- `s` is a state, 
- `delta` is a transition function

The function computes the set of all those states that can be reached from the state
`s` via $\varepsilon$-transitions.
Formally, the set $\texttt{epsClosure}(q)$ is defined inductively:
- $q \in \texttt{epsClosure}(q)$.
- $p \in \texttt{epsClosure}(q) \wedge r \in \delta(p, \varepsilon) \;\rightarrow\; r    \in \texttt{epsClosure}(q)$.
 
  If the state $p$ is an element of the $\varepsilon$-closure of the state $q$ 
  and there is an $\varepsilon$-transition from $p$ to some state $r$, then $r$ 
  is also an element of the $\varepsilon$-transition of $q$.
  
Instead of implementing the inductive definition directly we compute `epsClosure(s, delta)` via a fixpoint iteration.

In [3]:
def epsClosure(s, delta):
    Result = fixpoint({s}, lambda q: delta.get((q, ''), set()))
    return frozenset(Result)

In order to transform a non-deterministic <span style="font-variant:small-caps;">Fsm</span> into a deterministic 
<span style="font-variant:small-caps;">Fsm</span>
$\texttt{det}(A)$ we have to extend the function $\delta:Q \times \Sigma \rightarrow 2^Q$ into the function
$$\delta^*: Q \times \Sigma \rightarrow 2^Q. $$
The idea is that given a state $q$ and a character $c$,  $\delta^*(q,c)$ is the set of all states that the
<span style="font-variant:small-caps;">Fsm</span> $A$ could reach when it reads the character $c$ in state $q$ and then performs an arbitrary number of $\varepsilon$-transitions.  Formally, the definition of $\delta^*$ is as follows:
$$ \delta^*(q_1, c) := \bigcup \bigl\{ \texttt{ec}(q_2) \bigm| q_2 \in \delta(q_1, c) \bigr \}. $$
This formula is to be read as follows:
- For every state $q_2 \in Q$ that can be reached from the state $q_1$ by reading the character $c$ we
  compute the $\varepsilon$-closure $\texttt{ec}(q_2)$.
- Then we take the union of all these sets $\texttt{ec}(q_2)$.

The function $\delta^*$ is implemented as the function `deltaStar`, which takes three arguments:
- `s` is a state,
- `c` is a character,
- `𝛿` is a transition function.

This function computes the set of all those states that can be reached 
from `s` when we first have a transition from state `s` to some state `p` 
on reading the character `c` followed by any number of $\varepsilon$-transitions
starting in `p`.

In [4]:
def deltaStar(s, c, delta):
    return { p for q in delta.get((s, c), set())
               for p in epsClosure(q, delta)
           }

The function  $\delta^*$ maps a state into a set of states.  Since the <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(A)$ uses sets of states of the <span style="font-variant:small-caps;">Fsm</span> $A$ as its states we need a function that maps sets of states of the <span style="font-variant:small-caps;">Fsm</span> $A$ into sets of states.  Hence we generalize 
the function $\delta^*$ to the function
$$ \Delta: 2^Q \times \Sigma \rightarrow 2^Q $$
such that for a set $M$ of states and a character $c$ the expression $\Delta(M, c)$
computes the set of all those states that the <span style="font-variant:small-caps;">Fsm</span> $A$ could be in if it is in a state from the set $M$, then
reads the character $c$, and finally makes some $\varepsilon$-transitions.
The formal definition is as follows: 
$$ \Delta(M,c) := \bigcup \bigl\{ \delta^*(q,c) \bigm| q \in M \bigr\}. $$
This formula is easy to understand:  For every state  $q \in M$ we compute the set of states that the
<span style="font-variant:small-caps;">Fsm</span> could be in after reading the character $c$ and doing some 
$\varepsilon$-transitions.  Then we take the union of these sets.

In [5]:
def capitalDelta(M, c, delta):
    Result = { s for q in M 
                 for s in deltaStar(q, c, delta) 
             }
    return frozenset(Result)

Now we are ready to formally define how the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(A)$
is computed from the non-deterministic <span style="font-variant:small-caps;">Fsm</span>
$A := \bigl\langle Q, \Sigma, \delta, q_0, F \bigr\rangle$.
We define: 
$$ \texttt{det}(A) = \bigl\langle 2^Q, \Sigma, \Delta, \texttt{ec}(q_0), \widehat{F} \bigr\rangle $$
where the components of this tuple are given as follows:
- The set of states of $\texttt{det}(A)$ is the set of all subsets of $Q$ and therefore it is equal to the power set $2^Q$.

  Instead of computing the power set, the implementation iteratively computes all possible sets of states that the non-deterministic
  <span style="font-variant:small-caps;">Fsm</span> $A$ could be in.
- The input alphabet $\Sigma$ does not change when going from $A$ to $\texttt{det}(A)$.
  After all, the deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(A)$ has to recognize the same language as the non-deterministic
  <span style="font-variant:small-caps;">Fsm</span> $A$.
- The function $\Delta$ that has been defined previously specified how the set of states change when a
  character is read.
- The start state $\texttt{ec}(q_0)$ of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $\texttt{det}(A)$ is the set of all states
  that can be reached from the start state $q_0$ of the non-deterministic <span style="font-variant:small-caps;">Fsm</span> $A$
  via $\varepsilon$-transitions.
- The set of accepting states $\widehat{F}$ is the set of those subsets of $Q$ that contain an accepting
  state of the <span style="font-variant:small-caps;">Fsm</span> $Q$:
  $$\widehat{F} := \bigl\{ M \in 2^Q \mid M \cap F \not= \{\} \bigl\}. $$

In [ ]:
def nfa2dfa(nfa):
    States, Sigma, delta, q0, Final = nfa
    newStart   = epsClosure(q0, delta)
    nextStates = lambda M: { capitalDelta(M, c, delta) for c in Sigma }
    NewStates  = fixpoint({newStart}, nextStates)
    newDelta   = { (M, c): capitalDelta(M, c, delta) for M in NewStates
                                                     for c in Sigma
                 }
    NewFinal   = { M for M in NewStates 
                     if  M & Final != set() 
                 }
    return NewStates, Sigma, newDelta, newStart, NewFinal

To test this function, use the notebook `Test-NFA-2-DFA.ipynb`.